In [1]:
import hprof

In [2]:
x = hprof.open('./test/roc.dump')

UnexpectedEof: unterminated ascii string

In [5]:
help(hprof.heap)

Help on module hprof.heap in hprof:

NAME
    hprof.heap - Classes and functions implementing a Java-like object model.

CLASSES
    builtins.dict(builtins.object)
        Heap
    builtins.object
        JavaClassContainer
            JavaClassName
            JavaPackage
        JavaHierarchy
        JavaObject
            JavaArray
        Ref
    builtins.type(builtins.object)
        JavaClass
            JavaArrayClass
    
    class Heap(builtins.dict)
     |  A heap dump from an hprof file. An hprof file can technically contain
     |  several of these, but they usually don't.
     |  
     |  >>> heap, = hf.heaps
     |  >>> print(len(heap), 'objects')
     |  24465 objects
     |  
     |  If you happen to know the id of an object, you can get at it very quickly:
     |  
     |  >>> print(heap[0xce7e8000].make)
     |  Fånark
     |  
     |  Otherwise, you can find instances by class:
     |  
     |  >>> bikes = list(heap.exact_instances('com.example.cars.Bike'))
     |  >